This script gathers price data from Aave API
https://aave-api-v2.aave.com/

In [ ]:
pip install requests

In [1]:
import requests
import pandas as pd
import datetime
from datetime import timedelta
import time

In [2]:
def isnan(string):
    return string == 'NaN'

## V1

In [3]:
pool = '0x24a42fd28c976a61df5d00d0599c34c4f90748c8' #v1
date = datetime.datetime(2020, 12, 31)
date_string = date.strftime('%m-%d-%Y')
lst = []

In [4]:
def process_aave_v1(raw):
    for i in range(0,len(raw)):
        temp = []
        if raw[i]['symbol'] == 'USDC':
            if isnan(raw[i]['referenceItem']['liquidityRate']): #if item is empty, pool does not exists
                temp.append(['empty', 0, 0, 0, 0, 0, 0])
            else:
                epoch = int(raw[i]['referenceItem']['timestamp'])
                time = datetime.datetime.utcfromtimestamp(epoch)
                supply = float(raw[i]['referenceItem']['liquidityRate'])
                borrow = float(raw[i]['referenceItem']['variableBorrowRate'])
                supply_ma30 = float(raw[i]['avg30DaysLiquidityRate'])
                borrow_ma30 = float(raw[i]['avg30DaysVariableBorrowRate'])
                temp.append(['usdc', epoch, time, supply, borrow, supply_ma30, borrow_ma30])
                
        elif raw[i]['symbol'] == 'USDT':
            if isnan(raw[i]['referenceItem']['liquidityRate']): #if item is empty, pool does not exists
                temp.append(['empty', 0, 0, 0, 0, 0, 0])
            else:
                epoch = int(raw[i]['referenceItem']['timestamp'])
                time = datetime.datetime.utcfromtimestamp(epoch)
                supply = float(raw[i]['referenceItem']['liquidityRate'])
                borrow = float(raw[i]['referenceItem']['variableBorrowRate'])
                supply_ma30 = float(raw[i]['avg30DaysLiquidityRate'])
                borrow_ma30 = float(raw[i]['avg30DaysVariableBorrowRate'])
                temp.append(['usdt', epoch, time, supply, borrow, supply_ma30, borrow_ma30])
                
        elif raw[i]['symbol'] == 'DAI':
            if isnan(raw[i]['referenceItem']['liquidityRate']): #if item is empty, pool does not exists
                temp.append(['empty', 0, 0, 0, 0, 0, 0])
            else:
                epoch = int(raw[i]['referenceItem']['timestamp'])
                time = datetime.datetime.utcfromtimestamp(epoch)
                supply = float(raw[i]['referenceItem']['liquidityRate'])
                borrow = float(raw[i]['referenceItem']['variableBorrowRate'])
                supply_ma30 = float(raw[i]['avg30DaysLiquidityRate'])
                borrow_ma30 = float(raw[i]['avg30DaysVariableBorrowRate'])
                temp.append(['dai', epoch, time, supply, borrow, supply_ma30, borrow_ma30])
                
        elif raw[i]['symbol'] == 'ETH':
            if isnan(raw[i]['referenceItem']['liquidityRate']): #if item is empty, pool does not exists
                temp.append(['empty', 0, 0, 0, 0, 0, 0])
            else:
                epoch = int(raw[i]['referenceItem']['timestamp'])
                time = datetime.datetime.utcfromtimestamp(epoch)
                supply = float(raw[i]['referenceItem']['liquidityRate'])
                borrow = float(raw[i]['referenceItem']['variableBorrowRate'])
                supply_ma30 = float(raw[i]['avg30DaysLiquidityRate'])
                borrow_ma30 = float(raw[i]['avg30DaysVariableBorrowRate'])
                temp.append(['weth', epoch, time, supply, borrow, supply_ma30, borrow_ma30])
                
        elif raw[i]['symbol'] == 'WBTC':
            if isnan(raw[i]['referenceItem']['liquidityRate']): #if item is empty, pool does not exists
                temp.append(['empty', 0, 0, 0, 0, 0, 0])
            else:
                epoch = int(raw[i]['referenceItem']['timestamp'])
                time = datetime.datetime.utcfromtimestamp(epoch)
                supply = float(raw[i]['referenceItem']['liquidityRate'])
                borrow = float(raw[i]['referenceItem']['variableBorrowRate'])
                supply_ma30 = float(raw[i]['avg30DaysLiquidityRate'])
                borrow_ma30 = float(raw[i]['avg30DaysVariableBorrowRate'])
                temp.append(['wbtc', epoch, time, supply, borrow, supply_ma30, borrow_ma30])
                
        elif raw[i]['symbol'] == 'UNI':
            if isnan(raw[i]['referenceItem']['liquidityRate']): #if item is empty, pool does not exists
                temp.append(['empty', 0, 0, 0, 0, 0, 0])
            else:
                epoch = int(raw[i]['referenceItem']['timestamp'])
                time = datetime.datetime.utcfromtimestamp(epoch)
                supply = float(raw[i]['referenceItem']['liquidityRate'])
                borrow = float(raw[i]['referenceItem']['variableBorrowRate'])
                supply_ma30 = float(raw[i]['avg30DaysLiquidityRate'])
                borrow_ma30 = float(raw[i]['avg30DaysVariableBorrowRate'])
                temp.append(['uni', epoch, time, supply, borrow, supply_ma30, borrow_ma30])
                
        elif raw[i]['symbol'] == 'LINK':
            if isnan(raw[i]['referenceItem']['liquidityRate']): #if item is empty, pool does not exists
                temp.append(['empty', 0, 0, 0, 0, 0, 0])
            else:
                epoch = int(raw[i]['referenceItem']['timestamp'])
                time = datetime.datetime.utcfromtimestamp(epoch)
                supply = float(raw[i]['referenceItem']['liquidityRate'])
                borrow = float(raw[i]['referenceItem']['variableBorrowRate'])
                supply_ma30 = float(raw[i]['avg30DaysLiquidityRate'])
                borrow_ma30 = float(raw[i]['avg30DaysVariableBorrowRate'])
                temp.append(['link', epoch, time, supply, borrow, supply_ma30, borrow_ma30])
                
        else:
            continue

        lst.append(temp)

            
       

In [5]:
cont = True

while cont:
    query = f'https://aave-api-v2.aave.com/data/liquidity/v1?poolId={pool}&date={date_string}'
    response = requests.get(query)
    raw = response.json()
    process_aave_v1(raw)
    date = (date - timedelta(days=1))
    date_string = date.strftime('%m-%d-%Y')
    print(date_string)
    time.sleep(3)


12-30-2020
12-29-2020
12-28-2020
12-27-2020
12-26-2020
12-25-2020
12-24-2020
12-23-2020
12-22-2020
12-21-2020
12-20-2020
12-19-2020
12-18-2020
12-17-2020
12-16-2020
12-15-2020
12-14-2020
12-13-2020
12-12-2020
12-11-2020
12-10-2020
12-09-2020
12-08-2020
12-07-2020
12-06-2020
12-05-2020
12-04-2020
12-03-2020
12-02-2020
12-01-2020
11-30-2020
11-29-2020
11-28-2020
11-27-2020
11-26-2020
11-25-2020
11-24-2020
11-23-2020
11-22-2020
11-21-2020
11-20-2020
11-19-2020
11-18-2020
11-17-2020
11-16-2020
11-15-2020
11-14-2020
11-13-2020
11-12-2020
11-11-2020
11-10-2020
11-09-2020
11-08-2020
11-07-2020
11-06-2020
11-05-2020
11-04-2020
11-03-2020
11-02-2020
11-01-2020
10-31-2020
10-30-2020
10-29-2020
10-28-2020
10-27-2020
10-26-2020
10-25-2020
10-24-2020
10-23-2020
10-22-2020
10-21-2020
10-20-2020
10-19-2020
10-18-2020
10-17-2020
10-16-2020
10-15-2020
10-14-2020
10-13-2020
10-12-2020
10-11-2020
10-10-2020
10-09-2020
10-08-2020
10-07-2020
10-06-2020
10-05-2020
10-04-2020
10-03-2020
10-02-2020
10-01-2020

KeyError: 0

In [6]:
uni = [item for sublist in lst[0::7] for item in sublist]
wbtc = [item for sublist in lst[1::7] for item in sublist]
link = [item for sublist in lst[2::7] for item in sublist]
dai = [item for sublist in lst[3::7] for item in sublist]
usdc = [item for sublist in lst[4::7] for item in sublist]
usdt = [item for sublist in lst[5::7] for item in sublist]
weth = [item for sublist in lst[6::7] for item in sublist]

final_lst = [uni, wbtc, link, dai, usdc, usdt, weth]

In [9]:
currencies = ['uni', 'wbtc', 'link', 'dai', 'usdc', 'usdt', 'weth']
for i in range(0, len(currencies)):
    df_aave = pd.DataFrame(final_lst[i], columns=['asset', 'epoch', 'timestamp', 'supply', 'borrow', 'supply_ma30', 'borrow_ma30'])
    df = df_aave.copy()
    #df = df.sort_values(by='timestamp', ascending=True)
    df = df.reset_index(drop=True)
    #store in csv
    df.to_csv(f'data/aave_{currencies[i]}_interest_v1.csv', index=False)


## V2

In [ ]:
pool = '0xb53c1a33016b2dc2ff3653530bff1848a515c8c5' #v2
date = datetime.datetime(2022, 8, 30)
date_string = date.strftime('%m-%d-%Y')

In [ ]:
lst = []

In [ ]:
def process_aave_v2(raw):
    for i in range(0,len(raw)):
        temp = []
        if raw[i]['symbol'] == 'USDC':
            epoch = int(raw[i]['referenceItem']['timestamp'])
            time = datetime.datetime.utcfromtimestamp(epoch)
            supply = float(raw[i]['referenceItem']['liquidityRate'])
            borrow = float(raw[i]['referenceItem']['variableBorrowRate'])
            supply_ma30 = float(raw[i]['avg30DaysLiquidityRate'])
            borrow_ma30 = float(raw[i]['avg30DaysVariableBorrowRate'])
            temp.append(['usdc', epoch, time, supply, borrow, supply_ma30, borrow_ma30])
        elif raw[i]['symbol'] == 'USDT':
            epoch = int(raw[i]['referenceItem']['timestamp'])
            time = datetime.datetime.utcfromtimestamp(epoch)
            supply = float(raw[i]['referenceItem']['liquidityRate'])
            borrow = float(raw[i]['referenceItem']['variableBorrowRate'])
            supply_ma30 = float(raw[i]['avg30DaysLiquidityRate'])
            borrow_ma30 = float(raw[i]['avg30DaysVariableBorrowRate'])
            temp.append(['usdt', epoch, time, supply, borrow, supply_ma30, borrow_ma30])
        elif raw[i]['symbol'] == 'DAI':
            epoch = int(raw[i]['referenceItem']['timestamp'])
            time = datetime.datetime.utcfromtimestamp(epoch)
            supply = float(raw[i]['referenceItem']['liquidityRate'])
            borrow = float(raw[i]['referenceItem']['variableBorrowRate'])
            supply_ma30 = float(raw[i]['avg30DaysLiquidityRate'])
            borrow_ma30 = float(raw[i]['avg30DaysVariableBorrowRate'])
            temp.append(['dai', epoch, time, supply, borrow, supply_ma30, borrow_ma30])
        elif raw[i]['symbol'] == 'WETH':
            epoch = int(raw[i]['referenceItem']['timestamp'])
            time = datetime.datetime.utcfromtimestamp(epoch)
            supply = float(raw[i]['referenceItem']['liquidityRate'])
            borrow = float(raw[i]['referenceItem']['variableBorrowRate'])
            supply_ma30 = float(raw[i]['avg30DaysLiquidityRate'])
            borrow_ma30 = float(raw[i]['avg30DaysVariableBorrowRate'])
            temp.append(['weth', epoch, time, supply, borrow, supply_ma30, borrow_ma30])
        elif raw[i]['symbol'] == 'WBTC':
            epoch = int(raw[i]['referenceItem']['timestamp'])
            time = datetime.datetime.utcfromtimestamp(epoch)
            supply = float(raw[i]['referenceItem']['liquidityRate'])
            borrow = float(raw[i]['referenceItem']['variableBorrowRate'])
            supply_ma30 = float(raw[i]['avg30DaysLiquidityRate'])
            borrow_ma30 = float(raw[i]['avg30DaysVariableBorrowRate'])
            temp.append(['wbtc', epoch, time, supply, borrow, supply_ma30, borrow_ma30])
        elif raw[i]['symbol'] == 'UNI':
            epoch = int(raw[i]['referenceItem']['timestamp'])
            time = datetime.datetime.utcfromtimestamp(epoch)
            supply = float(raw[i]['referenceItem']['liquidityRate'])
            borrow = float(raw[i]['referenceItem']['variableBorrowRate'])
            supply_ma30 = float(raw[i]['avg30DaysLiquidityRate'])
            borrow_ma30 = float(raw[i]['avg30DaysVariableBorrowRate'])
            temp.append(['uni', epoch, time, supply, borrow, supply_ma30, borrow_ma30])
        elif raw[i]['symbol'] == 'LINK':
            epoch = int(raw[i]['referenceItem']['timestamp'])
            time = datetime.datetime.utcfromtimestamp(epoch)
            supply = float(raw[i]['referenceItem']['liquidityRate'])
            borrow = float(raw[i]['referenceItem']['variableBorrowRate'])
            supply_ma30 = float(raw[i]['avg30DaysLiquidityRate'])
            borrow_ma30 = float(raw[i]['avg30DaysVariableBorrowRate'])
            temp.append(['link', epoch, time, supply, borrow, supply_ma30, borrow_ma30])
        else:
            continue
            
        lst.append(temp)

In [ ]:
cont = True

while cont:
    query = f'https://aave-api-v2.aave.com/data/liquidity/v2?poolId={pool}&date={date_string}'
    response = requests.get(query)
    raw = response.json()
    process_aave_v2(raw)
    date = (date - timedelta(days=1))
    date_string = date.strftime('%m-%d-%Y')
    print(date_string)
    time.sleep(3)


In [ ]:
uni = [item for sublist in lst[0::7] for item in sublist]
wbtc = [item for sublist in lst[1::7] for item in sublist]
link = [item for sublist in lst[2::7] for item in sublist]
dai = [item for sublist in lst[3::7] for item in sublist]
usdc = [item for sublist in lst[4::7] for item in sublist]
weth = [item for sublist in lst[5::7] for item in sublist]
usdt = [item for sublist in lst[6::7] for item in sublist]

final_lst = [uni, wbtc, link, dai, usdc, weth, usdt]

In [ ]:
currencies = ['uni', 'wbtc', 'link', 'dai', 'usdc', 'weth', 'usdt']
for i in range(0, len(currencies)):
    df_aave = pd.DataFrame(final_lst[i], columns=['asset', 'epoch', 'timestamp', 'supply', 'borrow', 'supply_ma30', 'borrow_ma30'])
    df = df_aave.copy()
    df = df.sort_values(by='timestamp', ascending=True)
    df = df.reset_index(drop=True)
    #store in csv
    df.to_csv(f'data/aave_{currencies[i]}_interest_v2.csv', index=False)
